# LorentzNet: 基于洛伦兹等变性的图神经网络用于高能喷注鉴别

## 概述
LorentzNet是由中科院、微软亚洲研究院和北京大学等单位的研究人员开发的基于洛伦兹等变性的图神经网络用于高能物理对撞机实验上的喷注鉴别。

本教程介绍了LorentzNet的研究背景和技术路径，并展示了如何通过MindSpore训练和快速推断模型。 更多信息可以在以下位置找到：[论文](https://link.springer.com/article/10.1007/JHEP07(2022)030)。

## 技术路径
解决这个问题的具体流程如下：

创建数据集

模型构建

损失函数

模型训练

## LorentzNet

LorentzNet是一种用于喷注标记（jet tagging）的洛伦兹群等变图神经网络（Graph Neural Network, GNN），它通过保持洛伦兹对称性来提高深度学习在粒子物理应用中的性能。该模型利用高效的Minkowski点积注意力机制来聚合粒子的四矢量信息，实验结果表明，LorentzNet在两个代表性的喷注标记基准测试中均实现了最佳的标记性能，并且相比于现有的最先进算法有显著提升。此外，LorentzNet在仅有数千个喷注的训练样本下就能达到高度竞争的性能，显示了其出色的泛化能力和样本效率。

下图展示了LorentzNet的架构(来源于原始论文)：

输入层：接收粒子的4-动量和相关标量信息。

Lorentz群等变块（LGEB）：核心组件，通过堆叠实现，处理节点嵌入和坐标嵌入，利用Minkowski点积注意力机制。

消息传递：基于Minkowski内积和外积，通过注意力机制聚合粒子间信息。

解码层：对最终的节点嵌入进行解码，使用平均池化和全连接层生成分类输出。

![LorentzNet](./images/LorenzNet.PNG)





In [1]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import context

from src.dataset import retrieve_dataloaders as loader
from src.model import LorentzNet
from src.train import train_loop, test_loop, forward_fn

context.set_context(mode=1, device_target="CPU")

### 创建数据集
quark-gluon数据集，利用energyflow下载

In [2]:
dataset, dataloaders = loader(32, 320)

### 模型构建：
参数定义：

n_scalar  : 输入节点的特征维度

n_hidden  : 隐藏层维度

n_class   : 分类标签数目

dropout   : dropout rate

n_layers  : LGEB模块数目

c_weight  : 模型超参数


In [3]:
model = LorentzNet(n_scalar = 8, n_hidden = 72, n_class = 2,
                       dropout = 0.2, n_layers = 6,
                       c_weight = 0.001)

### 损失函数
定义损失函数和优化器

In [4]:
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=0.0002, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

### 模型训练
训练模型并保存检查点，利用Mindspore Insight记录训练过程

In [ ]:
grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
print('Train')
for t in range(35):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, dataloaders['train'], loss_fn, grad_fn, optimizer, t, 'LorenzNet_training_CPU')
    print()
    test_loop(model, dataloaders['val'], loss_fn)

print('Test')
test_loop(model, dataloaders['test'], loss_fn)
print("Done!")